# Prediction demand of bikes at each bike station at different time of the day
index = start_date, start_station_id
time period = { period 1: 1-3, 
                period 2: 4-6, 
                period 3: 7-9, 
                period 4: 10-12, 
                period 5: 13-15, 
                period 6: 16-18, 
                period 7: 19-21,
                period 8: 22-24}
peak = {period 3, period 6}

### Load the packages

In [2]:
import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr  
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from datetime import datetime
from sklearn.model_selection import train_test_split
import math
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error, median_absolute_error
import xgboost as xgb

### Import the data

In [67]:
df = pd.read_csv("trip.csv")
weather = pd.read_csv("weather.csv")
stations = pd.read_csv("station.csv")

In [68]:
#Change duration from seconds to minutes
df.duration /= 60
#remove major outliers from the data; trips longer than 6 hours. This will remove less than 0.5% of the data.
df['duration'].quantile(0.995)
df = df[df.duration <= 360]

In [69]:
#Convert to datetime so that it can be manipulated more easily
df["start_date2"] = pd.to_datetime(df.start_date, format='%m/%d/%Y %H:%M').dt.date

In [70]:
df["start_time"]=pd.to_datetime(df.start_date, format='%m/%d/%Y %H:%M').dt.hour
df["start_time_period"]=df.start_time.map(lambda x: "period " + str((x+2)//3) )

In [71]:
df.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,start_date2,start_time,start_time_period
0,4576,1.050000,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127,2013-08-29,14,period 5
1,4607,1.166667,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138,2013-08-29,14,period 5
2,4130,1.183333,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214,2013-08-29,10,period 4
3,4251,1.283333,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060,2013-08-29,11,period 4
4,4299,1.383333,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103,2013-08-29,12,period 4


In [72]:
#Create the data frame that will be used for training,
df2 = df.groupby(['start_date2','start_station_id','start_time_period']).agg({"id": "count", "duration":"sum"}).reset_index()
df2.rename(columns = {'start_date2':'date', 'start_station_id':'station', 'id':'trips', }, inplace = True)
df2.head()

,date,station,start_time_period,trips,duration
0,2013-08-29,2,period 5,1,26.333333
1,2013-08-29,2,period 6,1,8.716667
2,2013-08-29,2,period 7,3,38.150000
3,2013-08-29,3,period 5,1,2.766667
4,2013-08-29,3,period 6,1,11.733333


In [73]:
df2.describe()

,station,trips,duration
count,161707.000000,161707.000000,161707.000000
mean,51.582529,4.123953,56.581931
std,21.262277,5.401237,104.266902
min,2.000000,1.000000,1.000000
25%,41.000000,1.000000,9.800000
50%,57.000000,2.000000,23.216667
75%,68.000000,5.000000,58.283333
max,84.000000,87.000000,2731.066667


In [175]:
train = df2.iloc[:,0:4]
train.head()
# this is the output expected from spark for trip data

,date,station,start_time_period,trips
0,2013-08-29,2,period 5,1
1,2013-08-29,2,period 6,1
2,2013-08-29,2,period 7,3
3,2013-08-29,3,period 5,1
4,2013-08-29,3,period 6,1


In [176]:
train.shape

(161707, 4)

## Weather data

In [177]:
weather = pd.read_csv("weather.csv")
weather.shape

(3665, 24)

In [178]:
weather.isnull().sum()

date                                 0
max_temperature_f                    4
mean_temperature_f                   4
min_temperature_f                    4
max_dew_point_f                     54
mean_dew_point_f                    54
min_dew_point_f                     54
max_humidity                        54
mean_humidity                       54
min_humidity                        54
max_sea_level_pressure_inches        1
mean_sea_level_pressure_inches       1
min_sea_level_pressure_inches        1
max_visibility_miles                13
mean_visibility_miles               13
min_visibility_miles                13
max_wind_Speed_mph                   1
mean_wind_speed_mph                  1
max_gust_speed_mph                 899
precipitation_inches                 1
cloud_cover                          1
events                            3143
wind_dir_degrees                     1
zip_code                             0
dtype: int64

In [179]:
weather.date = pd.to_datetime(weather.date, format='%m/%d/%Y')

In [180]:
#It seems we have one entry per zip code
#We need to map the stations to one of the five zip codes
weather.zip_code.unique()

array([94107, 94063, 94301, 94041, 95113])

In [181]:
weather.events.unique()

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'rain', 'Rain-Thunderstorm'], dtype=object)

In [182]:
weather.loc[weather.events == 'rain', 'events'] = "Rain"
weather.loc[weather.events.isnull(), 'events'] = "Normal"
events = pd.get_dummies(weather.events)
weather = weather.merge(events, left_index = True, right_index = True)
weather = weather.drop(['events'],1)

In [183]:
#For each value of max_wind, find the median max_gust and use that to fill the null values.
weather.loc[weather.max_gust_speed_mph.isnull(), 'max_gust_speed_mph'] = weather.groupby('max_wind_Speed_mph').max_gust_speed_mph.apply(lambda x: x.fillna(x.median()))

In [184]:
#Change this feature from a string to numeric.
#Use errors = 'coerce' because some values currently equal 'T' and we want them to become NAs.
weather.precipitation_inches = pd.to_numeric(weather.precipitation_inches, errors = 'coerce')

In [185]:
#Change null values to the median, of values > 0, because T, I think, means True. 
#Therefore we want to find the median amount of precipitation on days when it rained.
weather.loc[weather.precipitation_inches.isnull(), 
            'precipitation_inches'] = weather[weather.precipitation_inches.notnull()].precipitation_inches.median()

In [186]:
weather.isnull().sum()

date                               0
max_temperature_f                  4
mean_temperature_f                 4
min_temperature_f                  4
max_dew_point_f                   54
mean_dew_point_f                  54
min_dew_point_f                   54
max_humidity                      54
mean_humidity                     54
min_humidity                      54
max_sea_level_pressure_inches      1
mean_sea_level_pressure_inches     1
min_sea_level_pressure_inches      1
max_visibility_miles              13
mean_visibility_miles             13
min_visibility_miles              13
max_wind_Speed_mph                 1
mean_wind_speed_mph                1
max_gust_speed_mph                13
precipitation_inches               0
cloud_cover                        1
wind_dir_degrees                   1
zip_code                           0
Fog                                0
Fog-Rain                           0
Normal                             0
Rain                               0
R

In [187]:
# impute other missing value with median first (to be improved later)
weather.loc[weather.max_temperature_f.isnull(), 'max_temperature_f'] = weather[weather.max_temperature_f.notnull()].max_temperature_f.median()
weather.loc[weather.mean_temperature_f.isnull(), 'mean_temperature_f'] = weather[weather.mean_temperature_f.notnull()].mean_temperature_f.median()
weather.loc[weather.min_temperature_f.isnull(), 'min_temperature_f'] = weather[weather.min_temperature_f.notnull()].min_temperature_f.median()

weather.loc[weather.max_dew_point_f.isnull(), 'max_dew_point_f'] = weather[weather.max_dew_point_f.notnull()].max_dew_point_f.median()
weather.loc[weather.mean_dew_point_f.isnull(), 'mean_dew_point_f'] = weather[weather.mean_dew_point_f.notnull()].mean_dew_point_f.median()
weather.loc[weather.min_dew_point_f.isnull(), 'min_dew_point_f'] = weather[weather.min_dew_point_f.notnull()].min_dew_point_f.median()

weather.loc[weather.max_humidity.isnull(), 'max_humidity'] = weather[weather.max_humidity.notnull()].max_humidity.median()
weather.loc[weather.mean_humidity.isnull(), 'mean_humidity'] = weather[weather.mean_humidity.notnull()].mean_humidity.median()
weather.loc[weather.min_humidity.isnull(), 'min_humidity'] = weather[weather.min_humidity.notnull()].min_humidity.median()

weather.loc[weather.max_visibility_miles.isnull(), 'max_visibility_miles'] = weather[weather.max_visibility_miles.notnull()].max_visibility_miles.median()
weather.loc[weather.mean_visibility_miles.isnull(), 'mean_visibility_miles'] = weather[weather.mean_visibility_miles.notnull()].mean_visibility_miles.median()
weather.loc[weather.min_visibility_miles.isnull(), 'min_visibility_miles'] = weather[weather.min_visibility_miles.notnull()].min_visibility_miles.median()

weather.isnull().sum()

date                               0
max_temperature_f                  0
mean_temperature_f                 0
min_temperature_f                  0
max_dew_point_f                    0
mean_dew_point_f                   0
min_dew_point_f                    0
max_humidity                       0
mean_humidity                      0
min_humidity                       0
max_sea_level_pressure_inches      1
mean_sea_level_pressure_inches     1
min_sea_level_pressure_inches      1
max_visibility_miles               0
mean_visibility_miles              0
min_visibility_miles               0
max_wind_Speed_mph                 1
mean_wind_speed_mph                1
max_gust_speed_mph                13
precipitation_inches               0
cloud_cover                        1
wind_dir_degrees                   1
zip_code                           0
Fog                                0
Fog-Rain                           0
Normal                             0
Rain                               0
R

In [188]:
weather = weather.fillna(0)

In [189]:
weather.isnull().sum()

date                              0
max_temperature_f                 0
mean_temperature_f                0
min_temperature_f                 0
max_dew_point_f                   0
mean_dew_point_f                  0
min_dew_point_f                   0
max_humidity                      0
mean_humidity                     0
min_humidity                      0
max_sea_level_pressure_inches     0
mean_sea_level_pressure_inches    0
min_sea_level_pressure_inches     0
max_visibility_miles              0
mean_visibility_miles             0
min_visibility_miles              0
max_wind_Speed_mph                0
mean_wind_speed_mph               0
max_gust_speed_mph                0
precipitation_inches              0
cloud_cover                       0
wind_dir_degrees                  0
zip_code                          0
Fog                               0
Fog-Rain                          0
Normal                            0
Rain                              0
Rain-Thunderstorm           

In [190]:
# next step, map weather data to station and time period of the day
# then merge train with weather

## Station data

In [191]:
stations

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013
5,7,Paseo de San Antonio,37.333798,-121.886943,15,San Jose,8/7/2013
6,8,San Salvador at 1st,37.330165,-121.885831,15,San Jose,8/5/2013
7,9,Japantown,37.348742,-121.894715,15,San Jose,8/5/2013
8,10,San Jose City Hall,37.337391,-121.886995,15,San Jose,8/6/2013
9,11,MLK Library,37.335885,-121.885660,19,San Jose,8/6/2013


In [192]:
stations2 = stations.drop(['installation_date'],1)

In [193]:
train = pd.merge(train, stations, left_on=['station'], right_on=['id'])
train = train.drop(['id'], 1)

In [194]:
train

,date,station,start_time_period,trips,name,lat,long,dock_count,city,installation_date
0,2013-08-29,2,period 5,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,2013-08-29,2,period 6,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
2,2013-08-29,2,period 7,3,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
3,2013-08-30,2,period 3,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
4,2013-08-30,2,period 6,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
5,2013-08-30,2,period 7,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
6,2013-08-31,2,period 5,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
7,2013-08-31,2,period 6,3,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
8,2013-08-31,2,period 7,3,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
9,2013-09-01,2,period 4,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013


## Special Date

In [195]:
#Find all of the holidays during our time span
calendar = USFederalHolidayCalendar()
holidays = calendar.holidays(start=train.date.min(), end=train.date.max())
holidays = pd.to_datetime(holidays, format='%Y/%m/%d').date
holidays

array([datetime.date(2013, 9, 2), datetime.date(2013, 10, 14),
       datetime.date(2013, 11, 11), datetime.date(2013, 11, 28),
       datetime.date(2013, 12, 25), datetime.date(2014, 1, 1),
       datetime.date(2014, 1, 20), datetime.date(2014, 2, 17),
       datetime.date(2014, 5, 26), datetime.date(2014, 7, 4),
       datetime.date(2014, 9, 1), datetime.date(2014, 10, 13),
       datetime.date(2014, 11, 11), datetime.date(2014, 11, 27),
       datetime.date(2014, 12, 25), datetime.date(2015, 1, 1),
       datetime.date(2015, 1, 19), datetime.date(2015, 2, 16),
       datetime.date(2015, 5, 25), datetime.date(2015, 7, 3)], dtype=object)

In [196]:
#Find all of the business days in our time span
us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
business_days = pd.DatetimeIndex(start=train.date.min(), end=train.date.max(), freq=us_bd)
business_days = pd.to_datetime(business_days, format='%Y/%m/%d').date
business_days

array([datetime.date(2013, 8, 29), datetime.date(2013, 8, 30),
       datetime.date(2013, 9, 3), datetime.date(2013, 9, 4),
       datetime.date(2013, 9, 5), datetime.date(2013, 9, 6),
       datetime.date(2013, 9, 9), datetime.date(2013, 9, 10),
       datetime.date(2013, 9, 11), datetime.date(2013, 9, 12),
       datetime.date(2013, 9, 13), datetime.date(2013, 9, 16),
       datetime.date(2013, 9, 17), datetime.date(2013, 9, 18),
       datetime.date(2013, 9, 19), datetime.date(2013, 9, 20),
       datetime.date(2013, 9, 23), datetime.date(2013, 9, 24),
       datetime.date(2013, 9, 25), datetime.date(2013, 9, 26),
       datetime.date(2013, 9, 27), datetime.date(2013, 9, 30),
       datetime.date(2013, 10, 1), datetime.date(2013, 10, 2),
       datetime.date(2013, 10, 3), datetime.date(2013, 10, 4),
       datetime.date(2013, 10, 7), datetime.date(2013, 10, 8),
       datetime.date(2013, 10, 9), datetime.date(2013, 10, 10),
       datetime.date(2013, 10, 11), datetime.date(2013, 10,

In [197]:
train['business_day'] = train.date.isin(business_days)
train['holiday'] = train.date.isin(holidays)
#Convert True to 1 and False to 0
train.business_day = train.business_day.map(lambda x: 1 if x == True else 0)
train.holiday = train.holiday.map(lambda x: 1 if x == True else 0)
train.head()

,date,station,start_time_period,trips,name,lat,long,dock_count,city,installation_date,business_day,holiday
0,2013-08-29,2,period 5,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0
1,2013-08-29,2,period 6,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0
2,2013-08-29,2,period 7,3,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0
3,2013-08-30,2,period 3,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0
4,2013-08-30,2,period 6,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0


In [198]:
#Convert date to the important features, year, month, weekday (0 = Monday, 1 = Tuesday...)
train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.month
train['weekday'] = pd.to_datetime(train['date']).dt.weekday
train

,date,station,start_time_period,trips,name,lat,long,dock_count,city,installation_date,business_day,holiday,year,month,weekday
0,2013-08-29,2,period 5,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0,2013,8,3
1,2013-08-29,2,period 6,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0,2013,8,3
2,2013-08-29,2,period 7,3,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0,2013,8,3
3,2013-08-30,2,period 3,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0,2013,8,4
4,2013-08-30,2,period 6,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0,2013,8,4
5,2013-08-30,2,period 7,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,1,0,2013,8,4
6,2013-08-31,2,period 5,1,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,0,0,2013,8,5
7,2013-08-31,2,period 6,3,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,0,0,2013,8,5
8,2013-08-31,2,period 7,3,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,0,0,2013,8,5
9,2013-09-01,2,period 4,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013,0,0,2013,9,6


## Train the model

In [199]:
start_time_period = pd.get_dummies(train.start_time_period)
station = pd.get_dummies(train.station)
city = pd.get_dummies(train.city)

In [200]:
train = train.merge(start_time_period, left_index = True, right_index = True)
# train = train.merge(station, left_index = True, right_index = True)
train = train.merge(city, left_index = True, right_index = True)

In [201]:
train = train.drop(['start_time_period','station','city','lat','long','installation_date','name','date'],1)

In [203]:
train

,trips,dock_count,business_day,holiday,year,month,weekday,period 0,period 1,period 2,...,period 4,period 5,period 6,period 7,period 8,Mountain View,Palo Alto,Redwood City,San Francisco,San Jose
0,1,27,1,0,2013,8,3,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,1,27,1,0,2013,8,3,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,3,27,1,0,2013,8,3,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,2,27,1,0,2013,8,4,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,27,1,0,2013,8,4,0,0,0,...,0,0,1,0,0,0,0,0,0,1
5,1,27,1,0,2013,8,4,0,0,0,...,0,0,0,1,0,0,0,0,0,1
6,1,27,0,0,2013,8,5,0,0,0,...,0,1,0,0,0,0,0,0,0,1
7,3,27,0,0,2013,8,5,0,0,0,...,0,0,1,0,0,0,0,0,0,1
8,3,27,0,0,2013,8,5,0,0,0,...,0,0,0,1,0,0,0,0,0,1
9,2,27,0,0,2013,9,6,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [204]:
labels = train.iloc[:,0]
labels.head()

0    1
1    1
2    3
3    2
4    1
Name: trips, dtype: int64

In [205]:
features = train.iloc[:,1:]
features.head()

,dock_count,business_day,holiday,year,month,weekday,period 0,period 1,period 2,period 3,period 4,period 5,period 6,period 7,period 8,Mountain View,Palo Alto,Redwood City,San Francisco,San Jose
0,27,1,0,2013,8,3,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,27,1,0,2013,8,3,0,0,0,0,0,0,1,0,0,0,0,0,0,1
2,27,1,0,2013,8,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,27,1,0,2013,8,4,0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,27,1,0,2013,8,4,0,0,0,0,0,0,1,0,0,0,0,0,0,1


In [206]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state = 2)

In [207]:
#15 fold cross validation. Multiply by -1 to make values positive.
#Used median absolute error to learn how many trips my predictions are off by.

def scoring(clf):
    scores = cross_val_score(clf, X_train, y_train, cv=15, n_jobs=1, scoring = 'neg_median_absolute_error')
    print (np.median(scores) * -1)

In [212]:
rfr = RandomForestRegressor(n_estimators = 10, # to be tuned
                            min_samples_leaf = 3,
                            random_state = 2)
scoring(rfr)

1.14444659023


In [213]:
gbr = GradientBoostingRegressor(learning_rate = 0.12,
                                n_estimators = 150,
                                max_depth = 8,
                                min_samples_leaf = 1,
                                random_state = 2)
scoring(gbr)

KeyboardInterrupt: 

In [ ]:
dtr = DecisionTreeRegressor(min_samples_leaf = 3,
                            max_depth = 8,
                            random_state = 2)
scoring(dtr)

In [209]:
labels.describe()

count    161707.000000
mean          4.123953
std           5.401237
min           1.000000
25%           1.000000
50%           2.000000
75%           5.000000
max          87.000000
Name: trips, dtype: float64

In [210]:
labels.median()

2.0